## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ['EIN', 'NAME'])

In [5]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T17','T15','T29','T14','T25','T2','T12','T13','T9']
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# store value counts
value_counts = application_df['CLASSIFICATION'].value_counts()

# get the values as a list
unique_values = value_counts.index.tolist()

# get only the ones I want to get rid of
classifications_to_replace = unique_values[5:]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [30]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies = pd.get_dummies(application_df['APPLICATION_TYPE'])

class_dummies = pd.get_dummies(application_df['CLASSIFICATION'])

affil_dummies = pd.get_dummies(application_df['AFFILIATION'])

use_case_dummies = pd.get_dummies(application_df['USE_CASE'])

org_dummies = pd.get_dummies(application_df['ORGANIZATION'])

inc_dummies = pd.get_dummies(application_df['INCOME_AMT'])

consid_dummies = pd.get_dummies(application_df['SPECIAL_CONSIDERATIONS'])

applications_w_dummy_df = pd.concat([application_df, app_dummies, class_dummies, affil_dummies, use_case_dummies, org_dummies, consid_dummies, inc_dummies], axis = 1)

applications_w_dummy_df


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,Y,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,...,0,1,0,0,0,0,0,0,0,0
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,...,0,0,1,0,0,0,0,0,0,0
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,...,0,0,0,1,0,0,0,0,0,0
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,...,0,1,0,0,0,0,0,0,0,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,...,0,1,0,0,0,0,0,0,0,0


In [31]:
# list columns to drop from our new dataframe
drop_cols = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE',
       'ORGANIZATION', 'SPECIAL_CONSIDERATIONS','INCOME_AMT']

applications_w_dummy_df = applications_w_dummy_df.drop(drop_cols, axis = 1)

In [32]:
# Split our preprocessed data into our features and target arrays
y = applications_w_dummy_df['IS_SUCCESSFUL']
X = applications_w_dummy_df.drop('IS_SUCCESSFUL', axis = 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [33]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [34]:
# find the shape of our data so we know how many input_dim will be in our nn_model

X_train_scaled.shape

(25724, 43)

## Compile, Train and Evaluate the Model

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5981 (23.36 KB)
Trainable params: 5981 (23.36 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5701 - accuracy: 0.7183
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accuracy: 0.7283
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.7314
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7315
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7318
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5477 - accuracy: 0.7315
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7317
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5458 - accuracy: 0.7328
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5453 - accura

In [38]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5617 - accuracy: 0.7307 - 540ms/epoch - 2ms/step
Loss: 0.5616827011108398, Accuracy: 0.7307288646697998


In [40]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity.h5")